In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout, ConnectionConfig
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
			)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__

live = client.is_live()
connected = client.is_connected()

print(f"Weaviate client is ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"live: {live}")
print(f"Connected: {connected}")

In [ ]:
# Do Update Collection and add properties
from weaviate.classes.config import Property, DataType

movies_col= client.collections.get("<COLLECTION_NAME>")
movies_col.config.update(
    Property(
        name="<PROPERTY_NAME>",
        description="<UPDATE_DESCRIPTION>"
    )
)

In [ ]:
# Change configuration
from weaviate.classes.config import Reconfigure, ReplicationDeletionStrategy

def get_collection_names():
    collections = [
        '<COL>',
        '<COL>',
        '<COL>'
    ]
    return collections

# Function to update the configuration for all collections
def update_collections_config(client):
    collections = get_collection_names()
    for collection_name in collections:
        try:
            # Get the collection by name
            collection = client.collections.get(collection_name)
            
            # Update its configuration
            collection.config.update(
                replication_config=Reconfigure.replication(
                    deletion_strategy=ReplicationDeletionStrategy.DELETE_ON_CONFLICT
                )
            )
            print(f"Updated configuration for collection: {collection_name}")
        
        except Exception as e:
            print(f"Failed to update configuration for collection {collection_name}: {str(e)}")

update_collections_config(client)